In [23]:
import folium
import pandas as pd
import requests
import json
import webbrowser

In [44]:
# 검색어를 던져서 응답 데이터중에 주소, 위도, 경도 질의어 리턴 함수
def net_get_geo_coder(query):
    KAKAO_API_KEY='.................................'
    url = '''
    https://dapi.kakao.com/v2/local/search/keyword.json?query={0}&category_group_code=PO3
    '''.format(query)
    headers={'Authorization': 'KakaoAK {0}'.format(KAKAO_API_KEY)}
    # API 쿼리 수행
    res = requests.get(url, headers=headers)
    res = res.json()
    try:
        return [float(res['documents'][0]['y']), float(res['documents'][0]['x'])]
    except:
        pass
    
    # 데이터 추출 => 리턴 값이 하나 이상이면 튜플로 리턴이 된다!!!
    # 데이터 리턴 : 주소, 위도, 경도, 질의어 순으로 튜플형태로 리턴  

In [45]:
net_get_geo_coder('제주특별자치도 서귀포시 신중로 55')

[33.2556629640412, 126.510542736677]

In [46]:
filename = 'jeju.csv'
df = pd.read_csv(filename, encoding='CP949', header=0)

In [47]:
df.head(2)

지역   읍면동          전화번호                         주소  \
0  제주시  일도1동  064-728-4412  제주특별자치도 제주시 성지로 71 (일도일동)   
1  제주시  일도2동  064-728-4442  제주특별자치도 제주시 고마로 28 (일도이동)   

                             일반현황                       청사현황  \
0  인구 3146명 / 면적 0.33㎢ / 11통 61개반          지상 3층  / 연면적 585㎡   
1   인구 38400명 / 면적2.18㎢ / 48통326반  지상 2층·지하 1층 / 연면적 1307.01   

                         자치센터현황     데이터기준일자  Unnamed: 8  
0                개소일 2000-12-20  2019-06-20         NaN  
1  총면적 617.22㎡ / 개소일 2000-12-24  2019-06-20         NaN

In [48]:
df['주소']=df['주소'].str.split('(').str.get(0)

In [49]:
df.head(2)

지역   읍면동          전화번호                   주소  \
0  제주시  일도1동  064-728-4412  제주특별자치도 제주시 성지로 71    
1  제주시  일도2동  064-728-4442  제주특별자치도 제주시 고마로 28    

                             일반현황                       청사현황  \
0  인구 3146명 / 면적 0.33㎢ / 11통 61개반          지상 3층  / 연면적 585㎡   
1   인구 38400명 / 면적2.18㎢ / 48통326반  지상 2층·지하 1층 / 연면적 1307.01   

                         자치센터현황     데이터기준일자  Unnamed: 8  
0                개소일 2000-12-20  2019-06-20         NaN  
1  총면적 617.22㎡ / 개소일 2000-12-24  2019-06-20         NaN

In [50]:
df.drop(columns='Unnamed: 8', inplace=True)

In [51]:
map = folium.Map(location=[33.4995342412698, 126.531171087129], zoom_start=10)
for index, row in df.iterrows():
    #print(index,row)
    geoData = net_get_geo_coder(row['주소'])
    #print(geoData)
    if geoData != None:
        folium.Marker(geoData, popup=row['읍면동'], icon=folium.Icon(color='red')).add_to(map)

svFilename = 'jeju.html'

map.save(svFilename)
import webbrowser
webbrowser.open(svFilename)  

False

In [52]:
import folium,json
map_osm = folium.Map(location=[33.50704277172384,126.49273908075342],zoom_start=10)
rfile = open(f'jeju-municipalities-geo.json', 'r', encoding='utf-8').read()
jsonData = json.loads(rfile)
folium.GeoJson(jsonData, name='geojson').add_to(map_osm)
map_osm
#map_osm.save('./map.html')

In [53]:
df.head(2)

지역   읍면동          전화번호                   주소  \
0  제주시  일도1동  064-728-4412  제주특별자치도 제주시 성지로 71    
1  제주시  일도2동  064-728-4442  제주특별자치도 제주시 고마로 28    

                             일반현황                       청사현황  \
0  인구 3146명 / 면적 0.33㎢ / 11통 61개반          지상 3층  / 연면적 585㎡   
1   인구 38400명 / 면적2.18㎢ / 48통326반  지상 2층·지하 1층 / 연면적 1307.01   

                         자치센터현황     데이터기준일자  
0                개소일 2000-12-20  2019-06-20  
1  총면적 617.22㎡ / 개소일 2000-12-24  2019-06-20

In [54]:
df['인구']=df['일반현황'].str.split('/').str.get(0).str.split('(').str.get(0).str.strip().str.split(' ').str.get(-1).str.replace('명','').astype('int')

In [55]:
df.sample(5)

지역   읍면동          전화번호                         주소  \
30  서귀포시   표선면  064-760-4413  제주특별자치도 서귀포시 표선면 표선중앙로 74   
3    제주시  이도2동  064-755-2021        제주특별자치도 제주시 오복3길 8    
9    제주시   화북동  064-728-4682      제주특별자치도 제주시 진남로6길 17    
26  서귀포시   대정읍  064-760-4012  제주특별자치도 서귀포시 대정읍 하모중앙로 20   
31  서귀포시   송산동  064-760-4503        제주특별자치도 서귀포시 소암로 4    

                                        일반현황                           청사현황  \
30       인구 11167명 / 면적 135.172㎢ / 10개리 94개반                   지상 2 층 지하 1층   
3             인구 41428명 / 면적 5.4㎢ / 49통 323반     지하 1층·지상 3층 / 연면적 1223.52㎡   
9   인구 25559('17.5월 현재) / 면적8.28㎢ / 33통 205반      지하 1층·지상3층 / 연면적 2375.80㎡   
26        인구 16610명 / 면적 78.53㎢ / 23개리 240개반  지상 2층·지하 1층 / 연면적 986.95㎡(본관)   
31               인구 5207명 / 면적 4.8㎢ / 2통 11반        지상 3층 지하 1층 / 연면적 1077㎡   

                          자치센터현황     데이터기준일자     인구  
30     총면적 214㎡ / 개소일 2006-08-16  2019-06-20  11167  
3   총면적 388.9㎡  / 개소일 2000-10-05  2019-06-20  41428  
9       연면적 54㎡ / 개소일 2000-12-27  2019-06-20  25559  
26     총면적 149㎡ / 개소일 2006-08-29  2019-06-20  16610  
31   총면적 276.6㎡ / 개소일 2000-12-28  2019-06-20   5207

In [56]:
df['지역']=df['지역'].replace(['제주시','서귀포시'],['Jeju','Seogwipo'])

In [57]:
df1=df.groupby(['지역']).sum()

In [58]:
df1

인구
지역              
Jeju      462396
Seogwipo  167641

In [59]:
import folium,json
map_osm = folium.Map(location=[33.50704277172384,126.49273908075342],zoom_start=10)
rfile = open('jeju-municipalities-geo.json', 'r', encoding='utf-8').read()
jsonData = json.loads(rfile)
folium.GeoJson(jsonData, name='geojson').add_to(map_osm)


In [60]:
map_osm.choropleth(geo_data=jsonData, 
               data=df1['인구'], 
               columns=[df1.index, df1['인구']], 
               key_on='feature.id',
               fill_color='PuRd',  #puRd, YlGnBu
               legend_name='제주 인구')
map_osm
#map_osm.save('./map.html')

In [61]:
import folium,json
map_osm = folium.Map(location=[33.50704277172384,126.49273908075342],zoom_start=10)
rfile = open('jeju.json', 'r', encoding='utf-8').read()
jsonData = json.loads(rfile)
folium.GeoJson(jsonData, name='geojson').add_to(map_osm)

df=df.groupby(['읍면동']).sum()
df.shape

(43, 1)

In [62]:
map_osm.choropleth(geo_data=jsonData, 
               data=df['인구'], 
               columns=[df.index, df['인구']], 
               key_on='feature.id',
               fill_color='YlGn', legend_name='제주 인구')
map_osm